In [1]:
import os
import re
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
# tokenizer = TweetTokenizer()


[nltk_data] Downloading package stopwords to /home/erica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
VECTOR_SIZE = 300

stopword_list = stopwords.words("english")
print(stopword_list)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
def get_cutted_sentences(raw_lines):
    sentences = []
    for line in raw_lines:
        line = line.strip()
        line = line.replace(" ' ", "'")
        line = re.sub("[^a-zA-Z']", " ", line)

        words = line.lower().split()
        words = [word for word in words if word not in stopword_list and len(word)>1]
        sentences.append(words)
        
    print(len(sentences))
    return sentences
    

In [4]:
no_labeled_path = os.path.join(os.getcwd(), "data", "training_nolabel.csv")
total_sentences = None

with open(no_labeled_path, 'r') as no_labeled_file:
    total_sentences = get_cutted_sentences(no_labeled_file.readlines())


1178614


In [5]:
# # build word2vec
# # sg=0 CBOW ; sg=1 skip-gram
# model = word2vec.Word2Vec(size=VECTOR_SIZE, min_count=30, window=7, sg=1)
# model.build_vocab(sentences)


In [6]:
# # train word2vec model ; shuffle data every epoch
# WORD2VEC_TRAINING_TIMES = 20
# for _ in range(WORD2VEC_TRAINING_TIMES):
#     random.shuffle(sentences)
#     model.train(sentences, total_examples=len(sentences), epochs=1)
    

In [7]:
# saved_folder = os.path.join(os.getcwd(), "saved_model")
# if not os.path.exists(saved_folder):
#     os.makedirs(saved_folder)

# model.save(os.path.join(os.getcwd(), "saved_model", 'dimension_300_window_7_skip_gram'))


In [8]:
word2vec_model_path = os.path.join(os.getcwd(), "saved_model", 'dimension_300_window_7_skip_gram')
word2vec_model = word2vec.Word2Vec.load(word2vec_model_path)

print(word2vec_model['bye'])
print(word2vec_model.most_similar('fever'))

/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[ 0.03549091  0.08357497  0.02273213  0.2484157   0.06983647 -0.05439604
 -0.07452954 -0.28196386 -0.08864312 -0.46760035 -0.03336187  0.04060542
 -0.19226886 -0.17290267  0.31306562 -0.17559303  0.01503302 -0.04937536
  0.18731526 -0.13611847  0.1181233  -0.1360042   0.2720578  -0.82264704
  0.0115023  -0.2562093   0.2668844   0.05964758 -0.34070194 -0.1497924
  0.3708003   0.02961739 -0.09593108 -0.0475378  -0.10257836 -0.2504861
  0.13461295 -0.01126851 -0.1218561   0.07747226 -0.2618634   0.22613083
  0.08969725  0.12427152 -0.37542117  0.2501521  -0.0499243   0.12466338
 -0.01164663 -0.39300147 -0.05939501 -0.13669845 -0.05366972  0.16360885
 -0.3533009  -0.1434421   0.06425     0.02739011 -0.1084506   0.24151093
 -0.34176365 -0.00194857 -0.03965796  0.1459657   0.04448299 -0.14057057
  0.09534362 -0.21862091 -0.1830834  -0.21467595  0.1029019   0.06562733
 -0.39670658  0.24311852  0.17941189 -0.13037372  0.5111816   0.06188105
 -0.01294602 -0.26524657 -0.18183404 -0.26294202  0.0

/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


In [9]:
labeled_path = os.path.join(os.getcwd(), "data", "training_label.csv")
labeled_data = []

with open(labeled_path, 'r') as labeled_file:
    for line in labeled_file.readlines():
        (label, text) = line.split("+++$+++")
        labeled_data.append([label.strip(), text.strip()])

labeled_dataframe = pd.DataFrame(labeled_data, columns =['Label', 'Text']) 
labeled_dataframe.head()


,Label,Text
0,1,are wtf ... awww thanks !
1,1,leavingg to wait for kaysie to arrive myspacin...
2,0,i wish i could go and see duffy when she comes...
3,1,i know eep ! i can ' t wait for one more day ....
4,0,so scared and feeling sick . fuck ! hope someo...


In [10]:
testing_path = os.path.join(os.getcwd(), "data", "testing_data.csv")
testing_data = []

with open(testing_path, 'r') as testing_file:
    for line in testing_file.readlines()[1:]:
        line_split = line.split(",")
        testing_id = line_split[0]
        text = ",".join(line_split[1:])
        testing_data.append([testing_id.strip(), text.strip()])

testing_dataframe = pd.DataFrame(testing_data, columns =['Id', 'Text']) 
testing_dataframe.head()


,Id,Text
0,0,"my dog ate our dinner . no , seriously ... he ..."
1,1,omg last day sooon n of primary noooooo x im g...
2,2,stupid boys .. they ' re so .. stupid !
3,3,hi ! do u know if the nurburgring is open for ...
4,4,"having lunch in the office , and thinking of h..."


In [11]:
training_frame, validation_frame = train_test_split(labeled_dataframe, test_size=0.1, random_state=42)
print(training_frame['Label'].value_counts())
print(validation_frame['Label'].value_counts())


1    90014
0    89986
Name: Label, dtype: int64
1    10031
0     9969
Name: Label, dtype: int64


In [12]:
training_x = training_frame['Text'].tolist()
training_x = get_cutted_sentences(training_x)
print(len(training_x))
print(training_x[:20])

validation_x = validation_frame['Text'].tolist()
validation_x = get_cutted_sentences(validation_x)

testing_x = testing_dataframe['Text'].tolist()
testing_x = get_cutted_sentences(testing_x)


180000
180000
[['lost', 'aww'], ["i've", 'reading', 'news', 'seems', 'keep', 'getting', 'worse', 'terrible'], ["i'm", 'seriously', 'upset', 'sad', 'ipod'], ['headed', 'dha', 'school', 'see', 'dance', 'party', 'think', 'dhats', 'paqe', 'lol'], ['biceps', 'killing', "can't", 'go', 'gym', 'today'], ['sounds', 'good', 'close'], ['messing', 'myspace', 'twitter', 'yahoo', 'messenger', 'hanging', 'beka', 'go', 'dairy', 'queen'], ['people', 'follow'], ['oh', 'yes', 'much', 'consumption', 'ice', 'cream', 'feel', 'bit', 'sick', 'though'], ['thanks', 'jacey', 'omg', 'boone', 'died', 'wtf', 'expecting'], ['fun'], ['wish', 'even', 'gm', 'though', 'babe'], ['rearranged', 'room', 'looks', 'spacious'], ['funny', 'say', 'past', 'times', "i've", 'met', "i've", 'huge', 'pimple', 'chin', 'well', 'scars'], ['hospital', 'feel', 'bad', 'jeremys', 'car', 'really', 'hope', 'hes', 'ok'], ['acho', 'lidooooo'], ['munchin', 'yummii', 'white', 'choco', 'cookies', 'lushh', 'nandos', 'hot', 'extra', 'chicken', 'oh', 

In [13]:
max_length = max(len(x) for x in training_x)
print(max_length)

max_length = max(len(x) for x in validation_x)
print(max_length)

max_length = max(len(x) for x in testing_x)
print(max_length)


29
22
28


In [14]:
training_y = training_frame['Label'].as_matrix()
print(training_y.shape)
print(training_y[:20])

validation_y = validation_frame['Label'].as_matrix()


(180000,)
['0' '0' '0' '1' '0' '0' '1' '1' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0'
 '0' '0']


/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [15]:
# word2vec_model.wv.vocab.keys(), 所有可以轉成vector的字, length: 14386

In [16]:
# vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
# vectorizer = TfidfVectorizer(analyzer='word', min_df=10)
# matrix = vectorizer.fit_transform([word for sentence in total_sentences for word in sentence])
# tfidf_map = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
# print ('vocab size :', len(tfidf_map))

#Save the tfidf 
tfidf_map_name = "tfidf_map.pickle"
# with open(tfidf_map_name, "wb") as pickle_file:
#     pickle.dump(tfidf_map, pickle_file)
with open(tfidf_map_name, "rb") as pickle_file:
    tfidf_map = pickle.load(pickle_file)


In [17]:
def get_document_vector(tokens, vector_size=300, token_size=30):
    vector = np.zeros((token_size, vector_size))
    index = 0
    for word in tokens:
        try:
            # vector[index] = (word2vec_model[word].reshape((1, vector_size))) * tfidf_map[word] # combining w2v vectors with tfidf value of words in the tweet.
            vector[index] = (word2vec_model[word].reshape((1, vector_size))) 
            index += 1
        except KeyError: # handling the case where the token is not
#             print(word)
#             print(word2vec_model[word])
#             print(tfidf_map[word])
            
            continue
    return vector


In [18]:
training_vector_x = np.array([get_document_vector(documnet) for documnet in training_x])
print(training_vector_x.shape)
print(training_vector_x[0])

validation_vector_x = np.array([get_document_vector(documnet) for documnet in validation_x])
print(validation_vector_x.shape)

testing_vector_x = np.array([get_document_vector(documnet) for documnet in testing_x])
print(testing_vector_x.shape)


/home/erica/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


(180000, 30, 300)
[[ 0.18590826 -0.02307397  0.30161124 ... -0.0933855  -0.24173458
   0.18248731]
 [ 0.09081154  0.02723532  0.13087037 ... -0.12416029 -0.17585213
   0.01903093]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
(20000, 30, 300)
(200000, 30, 300)


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(training_vector_x.shape[1], training_vector_x.shape[2]),
               dropout=0.1, recurrent_dropout=0.1))
model.add(Dropout(0.2))
model.add(LSTM(48, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(48, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(48, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16, return_sequences=False))
model.add(Dropout(0.2))

# Fully connected layer
model.add(Dense(64, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

model_name = "multiple_lstm_without_tfidf_classifier.h5"
model_path = os.path.join(os.getcwd(), model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_acc', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_acc', patience=20, verbose=1)

model_history = model.fit(training_vector_x, training_y, validation_data=(validation_vector_x, validation_y), 
                          epochs=200, batch_size=50,
                          callbacks = [checkpoint, earlystop])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 64)            93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 48)            21696     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 48)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 48)            18624     
_________________________________________________________________
dropout_3 (Dropout)  

In [ ]:
training_acc = model_history.history['acc']
val_acc = model_history.history['val_acc']

plt.plot(training_acc, label="training_accuracy")
plt.plot(val_acc, label="validation_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Binary Accuracy")
plt.title("Accuracy Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
# from keras.models import load_model

# model_path = os.path.join(os.getcwd(), model_name)
# model  = load_model(model_path)


In [ ]:
result = model.predict(testing_vector_x)
print(result.shape)
print(result)
result = (result>0.5).astype(int)
result = result.reshape(-1)
print(result.shape)
print(result)


In [ ]:
result_frame = pd.DataFrame({
    'id': testing_dataframe['Id'].tolist(),
    'label': result })
print(result_frame.head())
output_path = os.path.join(os.getcwd(), "result", "multiple_lstm_without_tfidf.csv")
result_frame.to_csv(output_path)
